In [421]:
def read_file(file_name):
	with open(file_name, "r") as f:
		data = f.read().splitlines()
	return data

In [422]:
EXAMPLE_FILE = "example"
INPUT_FILE = "input"
directions = {(0,1): '>', (0,-1): '<', (1,0): 'v', (-1,0): '^'}

In [423]:
import re

def prepare_data(file_name):
	monkey_map = {}
	data = read_file(file_name)
	start = None
	for row, line in enumerate(data):
		if line == '':
			break
		for column, tile in enumerate(line):
			if tile != ' ':
				if start == None:
					start = (row, column)
				monkey_map[(row, column)] = tile
	path = data[-1]
	path_re = re.compile(r'[1-9][0-9]*|R|L')
	path = path_re.findall(path)
	width = max([y for _,y in monkey_map.keys()])
	height = max([x for x,_ in monkey_map.keys()])
	return monkey_map, path, start, width, height

monkey_map, path, start, width, height = prepare_data(EXAMPLE_FILE)

In [424]:
import re

path = "10R5L5R10L4R5L5"
path_re = re.compile(r'[1-9][0-9]*|R|L')
path_re.findall(path)

['10', 'R', '5', 'L', '5', 'R', '10', 'L', '4', 'R', '5', 'L', '5']

In [425]:
def turn(facing, direction):
	x, y = facing
	if direction == 'R':
		return (-1 * y, -1 * x) if x == 1 else (y, -1 * x)
	elif direction == 'L':
		return (y, x) if y == 0 else (-1 * y, x)
	else:
		raise ValueError(direction)

print(turn((0,1), 'R'))
print(turn((0,1), 'L'))
print(turn((1,0), 'R'))
print(turn((1,0), 'L'))
print(turn((-1,0), 'R'))
print(turn((-1,0), 'L'))
print(turn((0,-1), 'R'))
print(turn((0,-1), 'L'))

(1, 0)
(-1, 0)
(0, -1)
(0, 1)
(0, 1)
(0, -1)
(-1, 0)
(1, 0)


In [426]:
def print_map(monkey_map, width, height):
	rect = [[' ' for _ in range(width + 1)] for _ in range(height + 1)]
	print(len(rect), len(rect[0]))
	for (x, y), tile in sorted(monkey_map.items()):
		rect[x][y] = tile
	for line in rect:
		print(''.join(line))
print_map(monkey_map, width, height)

12 16
        ...#    
        .#..    
        #...    
        ....    
...#.......#    
........#...    
..#....#....    
..........#.    
        ...#....
        .....#..
        .#......
        ......#.


In [427]:
def get_tile_pos(monkey_map, width, height, pos, facing):
	x, y = pos
	if x > height:
		x = 0
	if y > width:
		y = 0
	if x < 0:
		x = height
	if y < 0:
		y = width
	pos = (x, y)
	if pos in monkey_map.keys():
		return pos
	pos = tuple(sum(t) for t in zip(pos, facing))
	while pos not in monkey_map.keys():
		pos = tuple(sum(t) for t in zip(pos, facing))
		x, y = pos
		if x > height:
			x = 0
		if y > width:
			y = 0
		if x < 0:
			x = height
		if y < 0:
			y = width
		pos = (x, y)
	return pos
	
print(get_tile_pos(monkey_map, width, height, (0, 11), (1, 0)))
print(get_tile_pos(monkey_map, width, height, (5, 12), (0, 1)))


(0, 11)
(5, 0)


In [428]:
def get_password(row, column, facing):
	if facing == (0, 1):
		facing = 0
	elif facing == (1, 0):
		facing = 1
	elif facing == (0, -1):
		facing = 2
	elif facing == (-1, 0):
		facing = 3
	return 1000 * (row + 1) + 4 * (column + 1) + facing

get_password(5, 7, (0, 1))

6032

In [430]:
def solve_part1(file_name):
	monkey_map, path, pos, width, height = prepare_data(file_name)
	direction = (0, 1)
	monkey_map[pos] = directions[direction]
	for instruction in path:
		if instruction.isnumeric():
			for _ in range(int(instruction)):
				new_pos = tuple(sum(t) for t in zip(pos, direction))
				new_pos = get_tile_pos(monkey_map, width, height, new_pos, direction)
				if monkey_map[new_pos] == '#':
					monkey_map[pos] = directions[direction]
					break
				monkey_map[new_pos] = directions[direction]
				pos = new_pos
		else:
			direction = turn(direction, instruction)
			monkey_map[pos] = directions[direction]
	return get_password(*pos, direction)
	
print(solve_part1(EXAMPLE_FILE))
print(solve_part1(INPUT_FILE))

6032
181128
